In [45]:
import pandas as pd
import requests
import json

pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

TRADUCTOR_SIGFE_SIGCOM = pd.read_excel('input\\relacion_sigfe_sigcom_cristian_GG.xlsx')
TRADUCTOR_SIGFE_SIGCOM['COD SIGFE'] = TRADUCTOR_SIGFE_SIGCOM['COD SIGFE'].str.replace("'", "", regex = False)

TICKET_MERCADO_PUBLICO = '7CA7E3D8-361B-415F-84EB-88C0B89838B5'

EXCEPCIONES_SIGFE = {
    '221299901601': 'FERNANDO BARAONA EN RRHH. ANEDIN EN GASTO GENERAL',
    '221299901602': 'Todo en RRHH',
    '221299900902': 'Cardiologia y Cardiocirugia en RRHH. UC Christus en gastos generales',
    '221299900201': 'M Meneses cargado en RRHH',
    '221299900202': 'J Andueza y Cardiocirugia en RRHH'
}

In [102]:
from time import sleep
from requests import ReadTimeout


class AnalizadorSIGCOM:
    def __init__(self):
        pass
    
    def correr_programa(self):
        estado_ej_presup, disponibilidad_devengo, formato_gg_sigcom = self.cargar_archivos_y_tratar_df()

        suma_gastos_ej_presup = self.obtener_suma_ej_presup_total(estado_ej_presup)
        suma_gg_por_sigfe_sin_rrhh, facturas_gg, facturas_rrhh = self.desglosar_gastos_generales_y_rrhh(suma_gastos_ej_presup, disponibilidad_devengo)

        detalle_facturas = self.obtener_detalle_facturas(facturas_gg)

        suma_gg_por_sigcom_sin_rrhh = suma_gg_por_sigfe_sin_rrhh.groupby('Cod. SIGCOM', dropna = False).sum()

        suma_rrhh = suma_rrhh.groupby('Principal')['Monto Vigente'].sum().reset_index()
        suma_rrhh['Principal'] = suma_rrhh['Principal'].str.split(n = 1)
        suma_rrhh['Rut'] = suma_rrhh['Principal'].str[0]
        suma_rrhh['Nombre'] = suma_rrhh['Principal'].str[1]
        suma_rrhh = suma_rrhh[['Rut', 'Nombre', 'Monto Vigente']]


        formato_rellenado = self.rellenar_formato(suma_gg_por_sigcom_sin_rrhh, formato_gg_sigcom)

        self.guardar_archivos(suma_gg_por_sigfe_sin_rrhh, suma_rrhh, suma_gg_por_sigcom_sin_rrhh, formato_rellenado, facturas_apiladas)

    def cargar_archivos_y_tratar_df(self):
        estado_ej_presup = pd.read_excel('input\\SA_EstadoEjecucionPresupuestaria.xls', header = 6)
        estado_ej_presup = estado_ej_presup[['Nivel', 'Concepto Presupuestario', 'Devengado']]
        estado_ej_presup['COD SIGFE'] = estado_ej_presup['Concepto Presupuestario'].str.split().str[0]
        estado_ej_presup = pd.merge(estado_ej_presup, TRADUCTOR_SIGFE_SIGCOM, how = 'inner', on = 'COD SIGFE')

        disponibilidad_devengo = df_devengos = pd.read_excel('input\\SA_DisponibilidadDevengoPresupuestario.xls', header = 5)
        disponibilidad_devengo = disponibilidad_devengo[['Titulo', 'Principal', 'Número Documento', 'Concepto Presupuestario', 'Monto Vigente']]
        disponibilidad_devengo['COD SIGFE'] = disponibilidad_devengo['Concepto Presupuestario'].str.split().str[0]
        disponibilidad_devengo = pd.merge(disponibilidad_devengo, TRADUCTOR_SIGFE_SIGCOM, how = 'inner', on = 'COD SIGFE')

        formato_gg_sigcom = pd.read_excel('input\\Formato 3_Gasto General 2022-10.xlsx')

        formato_gg_sigcom = formato_gg_sigcom.rename(columns = {'Unnamed: 0': 'Centros de costo'})
        formato_gg_sigcom = formato_gg_sigcom.set_index('Centros de costo')
        

        return estado_ej_presup, disponibilidad_devengo, formato_gg_sigcom   

    def obtener_suma_ej_presup_total(self, estado_ej_presup):
        suma_ej_presup = estado_ej_presup.groupby(['COD SIGCOM', 'ITEM SIGCOM', 'COD SIGFE'], dropna = False)['Devengado'].sum().to_frame()
        suma_ej_presup['Devengado_ej_presup_y_estado_devengo'] = suma_ej_presup['Devengado']

        suma_ej_presup = suma_ej_presup.rename(columns = {'Devengado': 'Devengado_ej_presup'}).reset_index()
        return suma_ej_presup

    def desglosar_gastos_generales_y_rrhh(self, suma_gastos_ej_presup, disponibilidad_devengo):
        print(f'Analizando excepciones en el estado de devengo \n')
        facturas_gg = pd.DataFrame()
        facturas_rrhh = pd.DataFrame()

        suma_gastos_ej_presup_desglosados = suma_gastos_ej_presup.copy()

        for codigo_sigfe_excepcion in EXCEPCIONES_SIGFE.keys():
            print(f'------- Se está analizando la excepcion: {codigo_sigfe_excepcion} -------\n')
            query_excepcion = disponibilidad_devengo.query('`COD SIGFE` == @codigo_sigfe_excepcion')

            if codigo_sigfe_excepcion == '221299901601':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('BARAONA')
            
            elif codigo_sigfe_excepcion == '221299901602':
                mask_a_rrhh = query_excepcion['Principal'].notna()
            
            elif codigo_sigfe_excepcion == '221299900902':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('CARDIOLOGIA') | (query_excepcion['Principal'].str.contains('CARDIOCIRUGIA'))

            elif codigo_sigfe_excepcion == '221299900201':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('MANUEL MENESES')
                
            elif codigo_sigfe_excepcion == '221299900202':
                mask_a_rrhh = query_excepcion['Principal'].str.contains('ANDUEZA') | query_excepcion['Principal'].str.contains('CARDIOCIRUGIA')
            
            df_a_rrhh = query_excepcion[mask_a_rrhh]
            df_a_gg = query_excepcion[~mask_a_rrhh]

            print(f'Las siguientes facturas irán a RRHH:\n{df_a_rrhh[["Titulo", "Monto Vigente"]].to_markdown()} \n\n'
                  f'Las siguientes facturas irán a GG:\n{df_a_gg[["Titulo", "Monto Vigente"]].to_markdown()}\n')

            valor_a_rrhh = df_a_rrhh['Monto Vigente'].sum()
            valor_a_gg = df_a_gg['Monto Vigente'].sum()

            print(f'El monto destinado a RRHH será de: {valor_a_rrhh} \n'
                  f'El monto destinado a GG será de: {valor_a_gg}\n')
                
            print(f'-----------------------------------------------------------\n')
    
            facturas_rrhh = pd.concat([facturas_rrhh, df_a_rrhh])
            facturas_gg = pd.concat([facturas_gg, df_a_gg])
            
            mask_excepcion = (suma_gastos_ej_presup_desglosados['COD SIGFE'] == codigo_sigfe_excepcion)

            suma_gastos_ej_presup_desglosados.loc[mask_excepcion, 'Devengado_ej_presup_y_estado_devengo'] = valor_a_gg
            suma_gastos_ej_presup_desglosados.loc[mask_excepcion, 'Devengado_desglosado_a_gg_estado_devengo'] = valor_a_gg
            suma_gastos_ej_presup_desglosados.loc[mask_excepcion, 'Devengado_desglosado_a_rrhh_estado_devengo'] = valor_a_rrhh

        return suma_gastos_ej_presup_desglosados, facturas_gg, facturas_rrhh

    def obtener_detalle_facturas(self, facturas_gg):
        '''
        Esta función permite obtener el detalle de cada factura involucrada en el gasto general del item SIGCOM.
        Para esto, toma los items presupuestarios involucrados en el gasto general y busca las facturas en la disponibilidad de devengo.
        '''
        print(f'Se buscarán las ordenes de compra de las siguientes facturas: \n\n')
        facturas_a_buscar = facturas_gg.query('not((`COD SIGCOM` == 92) or ' \
                                                  '(`COD SIGCOM` == 93) or ' \
                                                  '(`COD SIGCOM` == 100) or ' \
                                                  '(`COD SIGCOM` == 133) or' \
                                                  '(`COD SIGCOM` == 170))')
        

        cols_a_mostrar = ["Titulo", "Número Documento", \
                          "COD SIGCOM", "COD SIGFE"]
        facturas_formateadas = facturas_a_buscar[cols_a_mostrar].to_markdown()
        print(facturas_formateadas)

        mask_con_oc = facturas_a_buscar['Titulo'].str.contains('/')

        facturas_a_buscar['folio_oc'] = facturas_a_buscar[mask_con_oc]['Titulo'].str.split('/').str[3]

        print(f'\n\n------ Buscando las ordenes de compra... ------ \n\n')
        facturas_a_buscar['detalle_oc'] = facturas_a_buscar[mask_con_oc]['folio_oc'].apply(self.funcion_obtener_requests_mercado_publico)
        
        return facturas_a_buscar

    def funcion_obtener_requests_mercado_publico(self, orden_de_compra):
        
        orden_de_compra = orden_de_compra.strip()

        if orden_de_compra and '-' in orden_de_compra:
            print(f'Pidiendo la orden de compra: {orden_de_compra}')
            url_request = f"https://api.mercadopublico.cl/servicios/v1/publico/ordenesdecompra.json?codigo={orden_de_compra}&ticket={TICKET_MERCADO_PUBLICO}"

            try:
                response = requests.get(url_request)
                print(response.status_code)
                response = response.json()['Listado'][0]['Items']
                sleep(1.9)
            
            except Exception as e:
                print(e)
                response = None

            return response
    

    
    def rellenar_formato(self, suma_gg_por_sigcom_sin_rrhh, formato_sigcom_gg):
        columnas_antiguas = formato_sigcom_gg.columns
        formato_sigcom_gg.columns = formato_sigcom_gg.columns.str.split('-').str[0].astype(float)

        for codigo_gasto in suma_gg_por_sigcom_sin_rrhh.index:
            if codigo_gasto in formato_sigcom_gg.columns:
                valor_a_ingresar = suma_gg_por_sigcom_sin_rrhh.loc[codigo_gasto, 'Devengado_GG_consolidado']
                formato_sigcom_gg.loc['Valor General', codigo_gasto] = valor_a_ingresar

        formato_sigcom_gg.columns = columnas_antiguas
        formato_sigcom_gg = formato_sigcom_gg.reset_index()
        
        return formato_sigcom_gg
    
    def guardar_archivos(self, suma_gg_por_sigfe_sin_rrhh, suma_rrhh, suma_gg_por_sigcom_sin_rrhh, formato_rellenado, facturas_apiladas):
        suma_gg_por_sigfe_sin_rrhh = suma_gg_por_sigfe_sin_rrhh.reset_index()
        suma_gg_por_sigcom_sin_rrhh = suma_gg_por_sigcom_sin_rrhh.reset_index()
        suma_rrhh = suma_rrhh.reset_index()
        formato_rellenado = formato_rellenado.reset_index(drop = True)
        facturas_apiladas = facturas_apiladas.reset_index()

        with pd.ExcelWriter('output.xlsx') as writer:
            formato_rellenado.to_excel(writer, sheet_name = 'formato_listo', index = False)
            suma_gg_por_sigfe_sin_rrhh.to_excel(writer, sheet_name = 'suma_gg_por_sigfe', index = False)
            suma_gg_por_sigcom_sin_rrhh.to_excel(writer, sheet_name = 'suma_gg_por_sigcom', index = False)
            suma_rrhh.to_excel(writer, sheet_name = 'suma_rrhh', index = False)
            facturas_apiladas.to_excel(writer, sheet_name = 'facturas_apiladas', index = False)


In [103]:
objeto = AnalizadorSIGCOM()
objeto.correr_programa()

Analizando excepciones en el estado de devengo 

------- Se está analizando la excepcion: 221299901601 -------

Las siguientes facturas irán a RRHH:
| Titulo   | Monto Vigente   |
|----------|-----------------| 

Las siguientes facturas irán a GG:
|    | Titulo                                                                         |   Monto Vigente |
|---:|:-------------------------------------------------------------------------------|----------------:|
| 38 | FE / 156 / 76197538 / 1499-3021-SE22 / SOCIEDAD MEDICA Y ADMINISTRACION ANEDIN |        21130030 |
| 39 | FE / 158 / 76197538 / 1499-3321-SE22 / SOCIEDAD MEDICA Y ADMINISTRACION ANEDIN |        34600805 |

El monto destinado a RRHH será de: 0 
El monto destinado a GG será de: 55730835

-----------------------------------------------------------

------- Se está analizando la excepcion: 221299901602 -------

Las siguientes facturas irán a RRHH:
|    | Titulo                                                               |   Monto

KeyError: 'Cod. SIGCOM'